# Amazon Titanテキスト埋め込みモデルを使用して埋め込みベクトルを作成し、類似性テストを実行する
> *このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します*

## セットアップ

In [1]:
import json
import os
import sys

import boto3
import botocore

import numpy as np
from numpy.linalg import norm

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

modelId="amazon.titan-embed-text-v1"

## 各文の埋め込みベクトルを生成するための再利用可能なPython関数を定義する

In [2]:
def sen2vec(sentence):
    # Convert the given sentence to a vector representation using a text embedding model.  
    input_body = {"inputText": sentence}
    try:
        response = bedrock_client.invoke_model(
            body=json.dumps(input_body),
            modelId=modelId,
            accept="application/json",
            contentType="application/json",
            )
        response_body = json.loads(response.get("body").read())
        vector = np.array(response_body.get("embedding"))            
        return vector
    except Exception as e:
        print(e)

## 埋め込みベクトルを生成し、配列に格納する

In [ ]:
with open("documents-jp.txt") as doc:
    num_records = len(doc.readlines())
    
doc_array=np.empty(shape=(num_records), dtype="S255")
embed_array = np.zeros(shape=(num_records, 1536))
    
with open("documents-jp.txt") as doc:
    for num, line in enumerate(doc, 0):
      doc = line.strip('\n')
      doc_array[num] = doc.encode("UTF-8")
      embed_array[num] = sen2vec(doc)

print(embed_array.shape)
print(embed_array)

## 類似性検索を使用してインタラクティブにクエリする


行列ベクトルストアから関連ドキュメントを引き出すために類似性検索を使用するインタラクティブなクエリ応答ループを作成します。類似度のしきい値として0.5を使用して、幻影(ハルシネーション)を限定します。ドキュメントストアのコース名に関連するクエリは、成功した結果につながります。結果のいずれも0.5のしきい値を超えない場合、「わかりません」が応答になります。

In [ ]:
while True:
    query = input("\nクエリを入力 もしくは quit を入力し終了: ")
    if query == "quit":
        break
    else:
        embed_query = sen2vec(query)
        denominator = norm(embed_array, axis=1) * norm(embed_query)
        similarity = embed_array.dot(embed_query) / denominator
        max_value = max(similarity)
        max_value_index = similarity.argmax()
        print("\nクエリ: " + query)
        if max_value > 0.5:
            print("質問に回答できるドキュメント: " + doc_array[max_value_index].decode('UTF-8'))
        else:
            print("回答: " + "わかりません。")
        print("\nSimilarity vector used for document selection")
        print(similarity)